# Mass spectrometry data

The objective of this exercise is to read in raw peptide MSMS spectrum information and output a dataframe.
The .msp file can be downloaded [here](ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz).

The information in this ASCII based text file is organized spectrum by spectrum.
The first line per spectrum provides formatted like this:

&emsp;<code>Name: sequence/charge_nmods_collisionenergy</code>

followed by a comment section wich can be disregarded and the actual spectrum data which is tab-separated:

&emsp;<code>m/z&emsp;intensity&emsp;additional_info</code>

Spectra are separated by an empty line.

Code a function that returns two DataFrames or arrays containing the processed and filtered data. The first one should contain the spectrum information (n_spectra, n_m/z_features) and the second one the sequences per row (n_spectra).

Here are some general guidelines:

* The m/z values need to be binned to integer values (mathematically rounded), otherwise the dataframe size would get out of hand. This will allow for multiple values mapped to a single bin (e.g. if there are peaks at 145.1 and 145.2). Here, only the maximum of those peaks should be kept in the final dataframe.

* Rows that are all-zero should be dropped.

Your function should allow for selecting a range on the x-axis (m/z-range). All peaks outside of this range can be disregarded. Furthermore, only spectra within a set collision energy range and a maximum sequence length should be contained in the output dataframe.

The faster your function runs, the better. I will time them all in the end.

In [103]:
import numpy as np
import pandas as pd
import timeit
import re

In [109]:
def msp_to_df(
    input_file,
    max_seq_len=30,
    min_ce=36,
    max_ce=40,
    mz_min=135,
    mz_max=1400,
):
    """
    Function to read spectrum data from .msp file and convert to dataframe.
    Args:
        input_file (str): path to .msp file
        max_seq_len (int): maximum acceptable sequence length
        min_ce (int): minimum collision energy of spectra to be included in df
        max_ce (int): maximum collision energy of spectra to be included in df
        mz_min (int): lower boundary for m/z to be included in df
        mz_max (int): lower boundary for m/z to be included in df

    Returns:
        df (pd.DataFrame or np.array): spectrum information within defined parameters [n_spectra, n_features]
        seqs (pd.DataFrame or np.array): sequences
    """
    msp_data = pd.read_csv(input_file, header = None, sep = "\t", error_bad_lines=False, names = ["m/z", "intensity", "comment"])
    msp_data = msp_data.dropna()
    
    msp_data = msp_data[["m/z", "intensity"]]
    msp_data["m/z"] = pd.to_numeric(msp_data["m/z"]).round(0)
    
    # splitten, unique nach m/z, transponieren und wieder zusammenfassen, Erfolg
    df = pd.DataFrame()
    first_split = 0
    i = 0
    for splitpoint in msp_data.index[msp_data["m/z"].diff() < 0]:
        sub = msp_data.loc[first_split:splitpoint-1, :]
        first_split = splitpoint
        sub = sub.groupby("m/z")["intensity"].max()
        df = df.append(sub.T, ignore_index = True)   
        
        i += 1
        if i > 10:
            break
    df.fillna(0)
    with open("../data/cptac2_mouse_hcd_selected.msp") as seq:
        seq_reader = seq.readlines()
    to_keep = []
    for line in seq_reader:
        if line[0:4] == "Name":
            to_keep.append(re.search(r' (.*?)/', line).group(1))
    seqs = pd.DataFrame(to_keep)
    
    
    return df, seqs
msp_to_df("../data/cptac2_mouse_hcd_selected.msp")

(     166.0    167.0     169.0    170.0     171.0   172.0    173.0     175.0  \
 0   2077.0   5074.9  162229.0  17394.5   20147.0  2206.0  45255.8   36459.4   
 1      NaN   1269.1    4683.8      NaN       NaN     NaN      NaN    6148.8   
 2      NaN      NaN       NaN      NaN       NaN     NaN      NaN       NaN   
 3      NaN      NaN     968.6      NaN       NaN     NaN      NaN       NaN   
 4      NaN   2154.6   13623.6      NaN    3766.7     NaN   1415.0    3242.3   
 5   3264.4  18863.0   36467.0   1844.5    6207.3     NaN   1350.7    6574.5   
 6      NaN      NaN    1759.9      NaN    2395.4     NaN    611.9       NaN   
 7      NaN      NaN   25554.4   1344.1     694.7     NaN      NaN   10550.1   
 8      NaN      NaN   20705.6      NaN    1589.0     NaN      NaN    6352.1   
 9      NaN      NaN   26509.5      NaN    2615.0     NaN      NaN   20796.3   
 10     NaN  52019.3  435346.0  67780.8  175005.0     NaN  20738.9  224729.0   
 
       176.0   177.0  ...    305.0   3